In [1]:
from datasets import load_dataset

dataset = load_dataset("clue", "iflytek")

Found cached dataset clue (/home/mymusise/.cache/huggingface/datasets/clue/iflytek/1.0.0/e508b66266ba417d60e89ed8b167699cb4b56d3a2ead29b5667907d08069dbfc)
100%|██████████| 3/3 [00:00<00:00, 480.70it/s]


In [2]:
labels = set(dataset['train']['label'])

In [3]:
from transformers import AutoTokenizer


model_path = "/data2/CPM-distill/CPM-Generate-distill"

tokenizer = AutoTokenizer.from_pretrained(model_path)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CpmTokenizer'. 
The class this function is called from is 'XLNetTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
def token_map(examples):
    return tokenizer(examples["sentence"], padding="max_length", max_length=256, truncation=True, add_special_tokens=False, return_tensors='pt')

train_dataset = dataset['train'].map(token_map, batched=True)
eval_dataset = dataset['validation'].map(token_map, batched=True)

  0%|          | 0/13 [00:00<?, ?ba/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.453 seconds.
Prefix dict has been built successfully.
100%|██████████| 13/13 [00:11<00:00,  1.10ba/s]
Loading cached processed dataset at /home/mymusise/.cache/huggingface/datasets/clue/iflytek/1.0.0/e508b66266ba417d60e89ed8b167699cb4b56d3a2ead29b5667907d08069dbfc/cache-c493234470e2b929.arrow


In [5]:
train_dataset = train_dataset.shuffle(seed=42)
eval_dataset = eval_dataset.shuffle(seed=42)

Loading cached shuffled indices for dataset at /home/mymusise/.cache/huggingface/datasets/clue/iflytek/1.0.0/e508b66266ba417d60e89ed8b167699cb4b56d3a2ead29b5667907d08069dbfc/cache-627d043af51f2191.arrow


In [6]:
for d in train_dataset["input_ids"]:
    if max(d) >= tokenizer.vocab_size or len(d) != 256:
        print(max(d), len(d))

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=len(labels)+1)

Some weights of the model checkpoint at /data2/CPM-distill/CPM-Generate-distill were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at /data2/CPM-distill/CPM-Generate-distill and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# ensure some model padding token id
model.config.pad_token_id = tokenizer.pad_token_id

In [9]:
import numpy as np
import evaluate


metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
print(f"before train: {trainer.evaluate()}")
trainer.train()
print(f"after train: {trainer.evaluate()}")

The following columns in the evaluation set  don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2599
  Batch size = 8
100%|██████████| 325/325 [00:09<00:00, 32.83it/s]
The following columns in the training set  don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
/home/mymusise/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Run

before train: {'eval_loss': 5.318122386932373, 'eval_accuracy': 0.002308580223162755, 'eval_runtime': 9.9382, 'eval_samples_per_second': 261.515, 'eval_steps_per_second': 32.702}


  1%|          | 25/2277 [00:04<06:22,  5.89it/s]